In [ ]:
!pip install tweepy pandas


جمع البيانات من تويتر

In [ ]:
import tweepy
import pandas as pd

# -------- 1) Twitter API Authentication --------
bearer_token = "AAAAAAAAAAAAAAAAAAAAAMpF4QEAAAAA4C3hA1G%2FhaVaD5z8Ygd3UCkC1Z8%3D2yAo5a1LQV3pCtEhbptnDzQQ5qrAfaqz5LzLHUhRyh9xH60YDj"
client = tweepy.Client(bearer_token=bearer_token)

# -------- 2) Keywords --------
main_keywords = [
    "hate speech",
    "go back to your country",
    "immigrants",
    "refugees",
    "politicians are corrupt",
    "racism",
    "racial slurs",
    "white supremacy",
    "black people",
    "Asian people",
    "islamophobia",
    "anti-semitism",
    "religious hate",
    "Muslims",
    "Jews",
    "Christians",
    "misogyny"
]

sentiment_keywords = [
    "disgusting",
    "horrible",
    "unacceptable",
    "shameful",
    "evil"
]

all_keywords = main_keywords + sentiment_keywords

# -------- 3) Search Tweets --------
query = " OR ".join(all_keywords) + " -is:retweet lang:en"

tweets = client.search_recent_tweets(
    query=query,
    tweet_fields=["public_metrics", "created_at"],
    max_results=100
)

# -------- 4) Prepare Data --------
data = []
if tweets.data:
    for tweet in tweets.data:
        tweet_url = f"https://twitter.com/user/status/{tweet.id}"

        metrics = tweet.public_metrics
        engagement = metrics.get("like_count", 0) + metrics.get("reply_count", 0) + metrics.get("retweet_count", 0) + metrics.get("quote_count", 0)

        # الكلمات المفتاحية اللي ظهرت بالتويتة
        matched_keywords = [kw for kw in all_keywords if kw.lower() in tweet.text.lower()]

        data.append([
            "Twitter",        # المنصة
            tweet_url,        # URL
            engagement,       # التفاعل الكلي
            tweet.text,       # النص
            ", ".join(matched_keywords) if matched_keywords else "N/A"
        ])

# -------- 5) Save to CSV --------
df = pd.DataFrame(data, columns=["Platform", "URL", "Engagement", "Text", "Matched Keywords"])
df.to_csv("twitter_data.csv", index=False, encoding="utf-8")

print("save in the file", len(df))


save in the file 99


تحميل الملف وعرض اول خمس اسطر

In [ ]:
import pandas as pd

# تحميل ملف تويتر)
df = pd.read_csv("twitter_data.csv")

# عرض أول 5 صفوف
print(df.head())


  Platform                                                URL  Engagement  \
0  Twitter  https://twitter.com/user/status/19701368662560...         130   
1  Twitter  https://twitter.com/user/status/19701368661218...        1088   
2  Twitter  https://twitter.com/user/status/19701368656481...        1219   
3  Twitter  https://twitter.com/user/status/19701368656229...        1151   
4  Twitter  https://twitter.com/user/status/19701368655431...           0   

                                                Text Matched Keywords  
0  RT @MikeCrispi: LIVE on the ground outside the...              NaN  
1  RT @AdamMoczar: ⚡🇺🇲 Charlie Kirk: “Islam is NO...          Muslims  
2  RT @aldamu_jo: Israeli police clashed with ult...             Jews  
3  RT @AzatAlsalim: A Hijabi woman tells a 'Gay f...              NaN  
4  @ScotGovFM @JohnSwinney @PalMissionUK Absolute...         shameful  


بدينا عملية تنظيف البيانات او شيء طبقناه
1.   ازالة القيم المتكررة
2.   ازالة القيم الفارغة
3. اعادة الفهرسة

In [ ]:
# إزالة القيم المكررة
df = df.drop_duplicates(subset="Text")

# إزالة القيم الفارغة
df = df.dropna(subset=["Text"])

# إعادة الفهرسة
df = df.reset_index(drop=True)

print("after cleaning", df.shape)


after cleaning (90, 5)


اضفنا لابل باستخدام مكتبة Detoxify
وطبعنا اول ١٠ اعمدة

In [ ]:
!pip install detoxify

from detoxify import Detoxify

# نموذج الكشف عن خطاب الكراهية
model = Detoxify('original')

# نحسب احتمالية أن النص فيه "toxic/hate"
df["toxicity_score"] = df["Text"].apply(lambda x: model.predict(str(x))["toxicity"])

# نحولها إلى Label (1 = Hate Speech, 0 = Not Hate Speech)
df["Label"] = df["toxicity_score"].apply(lambda x: 1 if x > 0.5 else 0)

print(df[["Text", "toxicity_score", "Label"]].head(10))

                                                Text  toxicity_score  Label
0  RT @MikeCrispi: LIVE on the ground outside the...        0.020658      0
1  RT @AdamMoczar: ⚡🇺🇲 Charlie Kirk: “Islam is NO...        0.125268      0
2  RT @aldamu_jo: Israeli police clashed with ult...        0.013052      0
3  RT @AzatAlsalim: A Hijabi woman tells a 'Gay f...        0.426302      0
4  @ScotGovFM @JohnSwinney @PalMissionUK Absolute...        0.018147      0
5  RT @arthurwatkins: A lot of those that are say...        0.040267      0
6  Just listen to these vile evil men! https://t....        0.885213      1
7                @JoJoFromJerz Define "hate speech."        0.020169      0
8  RT @joshxhowie: Throwing chicken bones at a 77...        0.384480      0
9  Do you know why it’s called a daemon thread? A...        0.000682      0


حذفنا ال url من الكلام عشان نحوله الى كلين تكست

In [ ]:
import re

def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)

df['clean_text'] = df['Text'].apply(remove_urls)

شلنا اليوزر نيم عشان نسوي كلين تكست

In [ ]:
def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

df['clean_text'] = df['clean_text'].apply(remove_mentions)


شلنا الايموجي واي رموز لان ما نحتاجها

In [ ]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # وجوه ضاحكة
                           u"\U0001F300-\U0001F5FF"  # رموز وأيقونات
                           u"\U0001F680-\U0001F6FF"  # مركبات
                           u"\U0001F1E0-\U0001F1FF"  # أعلام
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['clean_text'] = df['clean_text'].apply(remove_emojis)


حولنا النصوص الى Lowercase

In [ ]:
df['clean_text'] = df['clean_text'].str.lower()


شلنا علامات الترقيم

In [ ]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['clean_text'] = df['clean_text'].apply(remove_punctuation)


إزالة الـ Stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['clean_text'] = df['clean_text'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


حسبنا عدد الكلمات
"طول النص "



In [ ]:
df['word_count'] = df['clean_text'].apply(lambda x: len(x.split()))


حسبنا عدد الكلمات السلبية

In [ ]:
negative_words = ['disgusting', 'horrible', 'ugly', 'stupid', 'hate']

def count_negative_words(text):
    return sum(word in negative_words for word in text.split())

df['negative_word_count'] = df['clean_text'].apply(count_negative_words)


نشوف لو فيه توازن مع الليبل

In [ ]:
print(df['Label'].value_counts())

Label
0    65
1    25
Name: count, dtype: int64


عمل Sampling إذا فيه انحياز

In [ ]:
from sklearn.utils import resample

# نفصل البيانات حسب الليبل
df_majority = df[df['Label'] == 0]
df_minority = df[df['Label'] == 1]

# نعمل oversampling للـ minority
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # سحب مع التكرار
                                 n_samples=len(df_majority),
                                 random_state=42)

# ندمج الجدولين
df_balanced = pd.concat([df_majority, df_minority_upsampled])

حفظنا البيانات بعد عملية التنظيف

In [ ]:
# حفظ البيانات الجديدة
df.to_csv("twitter_labeled.csv", index=False, encoding="utf-8")

print("Save")


Save
